# 텍스트 요약

### 문장 임베딩 기반 텍스트 랭크(TextRank Based on Sentence Embedding)
https://wikidocs.net/91051

##  1. 텍스트랭크(TextRank)
텍스트랭크 알고리즘에 대해서 이해하기 위해서, 텍스트랭크 알고리즘의 기반이 된 페이지랭크 알고리즘에 대해서 간단히 이해해보겠습니다. 페이지랭크 알고리즘은 검색 엔진에서 웹 페이지의 순위를 정하기 위해 사용되던 알고리즘입니다. 텍스트랭크 알고리즘은 페이지랭크를 기반으로 한 텍스트 요약 알고리즘입니다. 텍스트랭크에서 그래프의 노드들은 문장들이며, 각 간선의 가중치는 문장들 간의 유사도를 의미합니다.

In [228]:
#! pip install nltk

In [9]:
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
import re
from nltk.corpus import stopwords 

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/songhyejun/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [229]:
#! pip install konlpy

In [230]:
#! pip install tqdm

In [231]:
#! pip install gensim

In [9]:
import konlpy
from konlpy.tag import Okt
from tqdm import tqdm

from gensim.models import doc2vec
from gensim.models.doc2vec import TaggedDocument

In [10]:
import pandas as pd

텍스트 요약에 사용할 테니스 관련 기사를 데이터프레임에 저장합니다.

In [53]:
df = pd.read_csv('./Data/네이버_뉴스_코인_20220509_20220609.csv')

In [1]:
!pwd

/Users/songhyejun/웹과텍스트마이닝


## 2. 사전 훈련된 임베딩(Pre-trained Embedding)

사전 훈련된 FastText 다운로드

In [13]:
! pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 55.2 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.9.2-py2.py3-none-any.whl (213 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp38-cp38-macosx_12_0_arm64.whl size=331333 sha256=6bfdf51efe368e279b200a6177611114f3e675eee899f2dd6b7a06ec646dea6b
  Stored in directory: /Users/songhyejun/Library/Caches/pip/wheels/93/61/2a/c54711a91c418ba06ba195b1d78ff24fcaad8592f2a694ac94
Successfully built fasttext


In [1]:
# from gensim.models import fasttext

import fasttext.util

fasttext.util.download_model('ko', if_exists='ignore')  # korea 
ft = fasttext.load_model('cc.ko.300.bin')

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 (100.00%) [==================================================>]================================================> ]=================================================> ] ]=================================================> ]=================================================> ]=================================================> ]=================================================> ]=================================================> ]]=================================================> ]=================================================> ]=================================================> ]=================================================> ]=================================================> ]=================================================> ]=================================================> ]=================================================> ]=================================================> ]=================================================> ]=================================================>

In [49]:
ft.get_word_vector('나').shape

(300,)

## 3. 문장 임베딩(Sentence Embedding)

어떤 다수의 문장을 가지고 있다고 해봅시다. 그리고 이 문장들을 서로 비교하고 싶습니다. 만약, 문장들을 각 문장을 표현하는 고정된 길이의 벡터로 변환한다면 벡터 간 비교로 문장을 비교할 수 있을 것입니다. 각 문장을 문장 벡터로 변환하는 방법은 여러가지 방법이 존재하지만, 여기서는 가장 간단한 방법 한 가지를 소개하고자 합니다.

문장 벡터를 얻는 가장 간단한 방법은 문장에 존재하는 단어 벡터들의 평균을 구하는 것입니다. 이번 챕터에서는 앞에서 소개한 방법으로 다운로드한 사전 훈련된 임베딩을 사용합니다. 예를 들어 사전 훈련된 fasttext로부터 문장 벡터는 다음과 같이 얻을 수 있습니다.

현재 fasttext.get_word_vector에는 300차원의 fasttext 벡터들이 저장되어져 있습니다. 
OOV 문제. 즉, fasttext.get_word_vecotr에 존재하지 않는 단어가 문장에 존재할 경우 해당 단어의 임베딩값으로 사용할 300차원의 영벡터도 만들어둡니다.

In [43]:
embedding_dim = 300
zero_vector = np.zeros(embedding_dim)

아래 함수는 문장의 각 단어를 사전 훈련된 fasttext 벡터로 변환하면서, OOV 문제가 발생할 경우에는 해당 단어를 영벡터로 변환합니다. 그리고 이렇게 모인 단어 벡터들의 평균을 구하여 반환합니다.

In [46]:
# 단어 벡터의 평균으로부터 문장 벡터를 얻는다.
def calculate_sentence_vector(sentence):
    
    sentence_vec = []
    
    for word in sentence: 
        
        word_vec = ft.get_word_vector(word)
        
        if sum(word_vec)>=0.1:
            sentence_vec.append(word_vec)
        
        else: # fasttext에 단어가 존재하지 않는다고 가정(vector 값이 너무 작음)
            sentence_vec.append(zero_vector)
    
    print()
    return sum([word_vec for word_vec in sentence_vec]) / len(sentence)


만약 '나는 정말 너를 사랑해' 라는 문장 벡터의 값을 얻고싶다면 해당 문장을 calculate_sentence_vector 함수의 입력으로 사용하면 됩니다. 이렇게 반환된 벡터값의 크기는 300차원이 될 것입니다.

In [47]:
kor_sent = ['나는', '정말', '너를', '좋아해']
sentence_vector = calculate_sentence_vector(kor_sent)
print(len(sentence_vector))


300


## 4. 텍스트 랭크를 이용한 텍스트 요약

여기서는 앞서 사전 훈련된 fasttext를 다운로드 하였다는 가정 하에 진행합니다.

In [225]:
#! pip install matplotlib

In [226]:
#! pip install sklearn

In [227]:
#! pip install networkx

In [65]:
import numpy as np
import re
import pandas as pd
import matplotlib.pyplot as plt
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from urllib.request import urlretrieve
import zipfile
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx

단어 빈도수가 높은데 중요하지 않은 단어와 일반적으로 자주 사용되는 단어를 합쳐서 불용어 만들었습니다.


자주 사용되는 불용어 링크 : 
https://www.ranks.nl/stopwords/korean

단어 빈도수를 기반으로 불용어 찾기

In [ ]:
# # 모든 기사 하나의 문자열로 합치기
# all_text = ''
# for i in range(len(documents)):
#     all_text = all_text + documents[i]

# from konlpy.tag import Hannanum
# import pandas as pd
# import numpy as np

# hannanum = Hannanum()
# text_list = hannanum.nouns(all_text)

# # 빈도수 100개 이상인 단어 중에서 불용어 직접 제거
# word_list = pd.Series(text_list)
# result = word_list.value_counts()
# # result[0:200]

# '것 등 수 이 코 루 비트코 전 중 이날 테 경우 투 이후 때문 때 관련 대비 그 이상 기 3 원 블록체 해당 설명 이번 24시간 약 씨 오후 주요 유 현재 상황 올해 방식 0. 규모 선 거 5월 바 기존 뒤 한편 권 가운데 한 지 개 정도 초 일부 들 만 시 내용 시각 후 전체 0 내 데 점 두 사건 2 비중'

In [66]:
in_person_stopword = '것 등 수 이 코 루 비트코 전 중 이날 테 경우 투 이후 때문 때 관련 대비 그 이상 기 3 원 블록체 해당 설명 이번 24시간 약 씨 오후 주요 유 현재 상황 올해 방식 0. 규모 선 거 5월 바 기존 뒤 한편 권 가운데 한 지 개 정도 초 일부 들 만 시 내용 시각 후 전체 0 내 데 점 두 사건 2 비중'

korean_stopwords=' 아 휴 아이구 아이쿠 아이고 어 나 우리 저희 따라 의해 을 를 에 의 가 으로 로 에게 뿐이다 의거하여 근거하여 입각하여 기준으로 예하면 예를 들면 예를 들자면 저 소인 소생 저희 지말고 하지마 하지마라 다른 물론 또한 그리고 비길수 없다 해서는 안된다 뿐만 아니라 만이 아니다 만은 아니다 막론하고 관계없이 그치지 않다 그러나 그런데 하지만 든간에 논하지 않다 따지지 않다 설사 비록 더라도 아니면 만 못하다 하는 편이 낫다 불문하고 향하여 향해서 향하다 쪽으로 틈타 이용하여 타다 오르다 제외하고 이 외에 이 밖에 하여야 비로소 한다면 몰라도 외에도 이곳 여기 부터 기점으로 따라서 할 생각이다 하려고하다 이리하여 그리하여 그렇게 함으로써 하지만 일때 할때 앞에서 중에서 보는데서 으로써 로써 까지 해야한다 일것이다 반드시 할줄알다 할수있다 할수있어 임에 틀림없다 한다면 등 등등 제 겨우 단지 다만 할뿐 딩동 댕그 대해서 대하여 대하면 훨씬 얼마나 얼마만큼 얼마큼 남짓 여 얼마간 약간 다소 좀 조금 다수 몇 얼마 지만 하물며 또한 그러나 그렇지만 하지만 이외에도 대해 말하자면 뿐이다 다음에 반대로 반대로 말하자면 이와 반대로 바꾸어서 말하면 바꾸어서 한다면 만약 그렇지않으면 까악 툭 딱 삐걱거리다 보드득 비걱거리다 꽈당 응당 해야한다 에 가서 각 각각 여러분 각종 각자 제각기 하도록하다 와 과 그러므로 그래서 고로 한 까닭에 하기 때문에 거니와 이지만 대하여 관하여 관한 과연 실로 아니나다를가 생각한대로 진짜로 한적이있다 하곤하였다 하 하하 허허 아하 거바 와 오 왜 어째서 무엇때문에 어찌 하겠는가 무슨 어디 어느곳 더군다나 하물며 더욱이는 어느때 언제 야 이봐 어이 여보시오 흐흐 흥 휴 헉헉 헐떡헐떡 영차 여차 어기여차 끙끙 아야 앗 아야 콸콸 졸졸 좍좍 뚝뚝 주룩주룩 솨 우르르 그래도 또 그리고 바꾸어말하면 바꾸어말하자면 혹은 혹시 답다 및 그에 따르는 때가 되어 즉 지든지 설령 가령 하더라도 할지라도 일지라도 지든지 몇 거의 하마터면 인젠 이젠 된바에야 된이상 만큼 어찌됏든 그위에 게다가 점에서 보아 비추어 보아 고려하면 하게될것이다 일것이다 비교적 좀 보다더 비하면 시키다 하게하다 할만하다 의해서 연이서 이어서 잇따라 뒤따라 뒤이어 결국 의지하여 기대여 통하여 자마자 더욱더 불구하고 얼마든지 마음대로 주저하지 않고 곧 즉시 바로 당장 하자마자 밖에 안된다 하면된다 그래 그렇지 요컨대 다시 말하자면 바꿔 말하면 즉 구체적으로 말하자면 시작하여 시초에 이상 허 헉 허걱 바와같이 해도좋다 해도된다 게다가 더구나 하물며 와르르 팍 퍽 펄렁 동안 이래 하고있었다 이었다 에서 로부터 까지 예하면 했어요 해요 함께 같이 더불어 마저 마저도 양자 모두 습니다 가까스로 하려고하다 즈음하여 다른 다른 방면으로 해봐요 습니까 했어요 말할것도 없고 무릎쓰고 개의치않고 하는것만 못하다 하는것이 낫다 매 매번 들 모 어느것 어느 로써 갖고말하자면 어디 어느쪽 어느것 어느해 어느 년도 라 해도 언젠가 어떤것 어느것 저기 저쪽 저것 그때 그럼 그러면 요만한걸 그래 그때 저것만큼 그저 이르기까지 할 줄 안다 할 힘이 있다 너 너희 당신 어찌 설마 차라리 할지언정 할지라도 할망정 할지언정 구토하다 게우다 토하다 메쓰겁다 옆사람 퉤 쳇 의거하여 근거하여 의해 따라 힘입어 그 다음 버금 두번째로 기타 첫번째로 나머지는 그중에서 견지에서 형식으로 쓰여 입장에서 위해서 단지 의해되다 하도록시키다 뿐만아니라 반대로 전후 전자 앞의것 잠시 잠깐 하면서 그렇지만 다음에 그러한즉 그런즉 남들 아무거나 어찌하든지 같다 비슷하다 예컨대 이럴정도로 어떻게 만약 만일 위에서 서술한바와같이 인 듯하다 하지 않는다면 만약에 무엇 무슨 어느 어떤 아래윗 조차 한데 그럼에도 불구하고 여전히 심지어 까지도 조차도 하지 않도록 않기 위하여 때 시각 무렵 시간 동안 어때 어떠한 하여금 네 예 우선 누구 누가 알겠는가 아무도 줄은모른다 줄은 몰랏다 하는 김에 겸사겸사 하는바 그런 까닭에 한 이유는 그러니 그러니까 때문에 그 너희 그들 너희들 타인 것 것들 너 위하여 공동으로 동시에 하기 위하여 어찌하여 무엇때문에 붕붕 윙윙 나 우리 엉엉 휘익 윙윙 오호 아하 어쨋든 만 못하다 하기보다는 차라리 하는 편이 낫다 흐흐 놀라다 상대적으로 말하자면 마치 아니라면 쉿 그렇지 않으면 그렇지 않다면 안 그러면 아니었다면 하든지 아니면 이라면 좋아 알았어 하는것도 그만이다 어쩔수 없다 하나 일 일반적으로 일단 한켠으로는 오자마자 이렇게되면 이와같다면 전부 한마디 한항목 근거로 하기에 아울러 하지 않도록 않기 위해서 이르기까지 이 되다 로 인하여 까닭으로 이유만으로 이로 인하여 그래서 이 때문에 그러므로 그런 까닭에 알 수 있다 결론을 낼 수 있다 으로 인하여 있다 어떤것 관계가 있다 관련이 있다 연관되다 어떤것들 에 대해 이리하여 그리하여 여부 하기보다는 하느니 하면 할수록 운운 이러이러하다 하구나 하도다 다시말하면 다음으로 에 있다 에 달려 있다 우리 우리들 오히려 하기는한데 어떻게 어떻해 어찌됏어 어때 어째서 본대로 자 이 이쪽 여기 이것 이번 이렇게말하자면 이런 이러한 이와 같은 요만큼 요만한 것 얼마 안 되는 것 이만큼 이 정도의 이렇게 많은 것 이와 같다 이때 이렇구나 것과 같이 끼익 삐걱 따위 와 같은 사람들 부류의 사람들 왜냐하면 중의하나 오직 오로지 에 한하다 하기만 하면 도착하다 까지 미치다 도달하다 정도에 이르다 할 지경이다 결과에 이르다 관해서는 여러분 하고 있다 한 후 혼자 자기 자기집 자신 우에 종합한것과같이 총적으로 보면 총적으로 말하면 총적으로 대로 하다 으로서 참 그만이다 할 따름이다 쿵 탕탕 쾅쾅 둥둥 봐 봐라 아이야 아니 와아 응 아이 참나 년 월 일 령 영 일 이 삼 사 오 육 륙 칠 팔 구 이천육 이천칠 이천팔 이천구 하나 둘 셋 넷 다섯 여섯 일곱 여덟 아홉 령 영 이 있 하 것 들 그 되 수 이 보 않 없 나 사람 주 아니 등 같 우리 때 년 가 한 지 대하 오 말 일 그렇 위하 때문 그것 두 말하 알 그러나 받 못하 일 그런 또 문제 더 사회 많 그리고 좋 크 따르 중 나오 가지 씨 시키 만들 지금 생각하 그러 속 하나 집 살 모르 적 월 데 자신 안 어떤 내 내 경우 명 생각 시간 그녀 다시 이런 앞 보이 번 나 다른 어떻 여자 개 전 들 사실 이렇 점 싶 말 정도 좀 원 잘 통하 놓'

stop_words = in_person_stopword + korean_stopwords

stop_words = stop_words.split(' ')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/songhyejun/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/songhyejun/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


해당 기사를 문장 토큰화한 결과를 저장한 sentences라는 열을 새로 만듭니다.

In [241]:
df['sentences'] = df['Article'].apply(sent_tokenize)

토큰화와 전처리를 위한 함수들을 정의합니다.

In [68]:
# from nltk.tokenize import word_tokenize 

# 토큰화 함수 
def tokenization(sentences):
    return [word_tokenize(sentence) for sentence in sentences]

# 전처리 함수
def preprocess_sentence(sentence):
  # 영어를 제외한 숫자, 특수 문자 등은 전부 제거. 모든 알파벳은 소문자화
  sentence = [re.sub(r'[^a-zA-z\s]', '', word).lower() for word in sentence]
  # 불용어가 아니면서 단어가 실제로 존재해야 한다.
  return [word for word in sentence if word not in stop_words and word]

# 위 전처리 함수를 모든 문장에 대해서 수행. 이 함수를 호출하면 모든 행에 대해서 수행.
def preprocess_sentences(sentences):
    return [preprocess_sentence(sentence) for sentence in sentences]

문장 토큰화를 진행한 'sentences'열에 대해서 단어 토큰화와 전처리를 적용한 'tokenized_sentences' 열을 새로 만듭니다.

In [245]:
df['tokenized_sentences'] = df['sentences'].apply(tokenization)
df['tokenized_sentences'] = df['tokenized_sentences'].apply(preprocess_sentences)

df[['Article', 'sentences','tokenized_sentences']].head()

,Article,sentences,tokenized_sentences
0,"권씨 트위터 비공개 전환…""이제 루나 3.0 나오나"" 비판도국내 투자자들 '사기 ...","[ 권씨 트위터 비공개 전환…""이제 루나 3.0 나오나"" 비판도국내 투자자들 '사기...","[[lunc, [, ], lunc, luna], [], [], [], [], [lu..."
1,거래소 독과점 해소·실명계좌 확대 등 다른 현안도 공동 대응 KDA...,[ 거래소 독과점 해소·실명계좌 확대 등 다른 현안도 공동 대응 KD...,"[[kda], [[, kda, ], kda], [], [btx], [kda], []..."
2,기사내용 요약비트코인 3800만원대·이더리움 220만원대오는 10일 미국 5월 소...,[ 기사내용 요약비트코인 3800만원대·이더리움 220만원대오는 10일 미국 5월 ...,"[[[, ], cpi], [], [], [], [], [cpi], [cpi], [n..."
3,"코빗 리서치센터 분석…""발행사가 안정성 지속 개선해야""스테이블코인의 안정성을 판단...","[ 코빗 리서치센터 분석…""발행사가 안정성 지속 개선해야""스테이블코인의 안정성을 판...","[[usd, ust], [], [], [ust], [], [], [], [], []..."
4,"빗썸경제연구소, 첫 리포트 발행""2024~2028년 비트코인 가격, 최저 6000...","[ 빗썸경제연구소, 첫 리포트 발행""2024~2028년 비트코인 가격, 최저 600...","[[], [news, btc], [], [], [], [], [], [], [], ..."


단어 벡터의 평균을 구하는 함수를 정의합니다. 단, 문장 길이가 0일 경우에는 300차원의 영벡터를 리턴합니다. 현재 불용어를 제거하였기 때문에 문장의 모든 단어가 불용어인 경우에는 길이가 0인 문장이 생길 수 있기 때문입니다.

In [70]:
# 단어 벡터의 평균으로부터 문장 벡터를 얻는다.
def calculate_sentence_vector(sentence):
    
    sentence_vec = []
    
    if len(sentence) != 0:
        for word in sentence: 

            word_vec = ft.get_word_vector(word)

            if sum(word_vec)>=0.1:
                sentence_vec.append(word_vec)

            else: # fasttext에 단어가 존재하지 않는다고 가정(vector 값이 너무 작음)
                sentence_vec.append(zero_vector)

        print()
        return sum([word_vec for word_vec in sentence_vec]) / len(sentence)
    
    
    else: 
        return zero_vector

이를 모든 행에 대해서 수행하기 위해서 모든 문장에 대해서 수행하는 함수를 정의합니다.

In [71]:
# 각 문장에 대해서 문장 벡터를 반환
def sentences_to_vectors(sentences):
    return [calculate_sentence_vector(sentence) 
              for sentence in sentences]

모든 문장에 대해서 문장 벡터를 만듭니다.

In [224]:
# 실제는 이 코드를 실행해서 SentenceEmbedding 열을 만들어야 하지만 결과가 너무 길게 출력되어서 문자열로 일단 변경해놓음.

"df['SentenceEmbedding'] = df['tokenized_sentences'].apply(sentences_to_vectors)"


"df['SentenceEmbedding'] = df['tokenized_sentences'].apply(sentences_to_vectors)"

문장 벡터들 간의 코사인 유사도를 구한 유사도 행렬을 만듭니다. 이 유사도 행렬의 크기는 (문장 개수 × 문장 개수)입니다.

In [73]:
def similarity_matrix(sentence_embedding):
    sim_mat = np.zeros([len(sentence_embedding), len(sentence_embedding)])
    for i in range(len(sentence_embedding)):
        for j in range(len(sentence_embedding)):
            sim_mat[i][j] = cosine_similarity(sentence_embedding[i].reshape(1, embedding_dim),
                                          sentence_embedding[j].reshape(1, embedding_dim))[0,0]
    return sim_mat

이 결과를 저장한 'SimMatrix'열을 만듭니다.

In [74]:
df['SimMatrix'] = df['SentenceEmbedding'].apply(similarity_matrix)
df['SimMatrix']

0      [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
1      [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0....
2      [[1.0000000000000002, 0.0, 0.0, 0.0, 0.0, 1.00...
3      [[0.9999998807907104, 0.0, 0.0, 0.736306428909...
4      [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
                             ...                        
555    [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
556    [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
557    [[0.9999998807907104, 0.0, 0.0, 0.0, 0.2798353...
558    [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
559    [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
Name: SimMatrix, Length: 560, dtype: object

두번째 샘플을 기준으로 지금까지 만든 열들의 크기를 확인해봅시다.

In [75]:
print('두번째 샘플의 문장 개수 :',len(df['tokenized_sentences'][1]))
print('두번째 샘플의 문장 벡터가 모인 문장 행렬의 크기(shape) :',np.shape(df['SentenceEmbedding'][1]))
print('두번째 샘플의 유사도 행렬의 크기(shape) :',df['SimMatrix'][1].shape)

두번째 샘플의 문장 개수 : 7
두번째 샘플의 문장 벡터가 모인 문장 행렬의 크기(shape) : (7, 300)
두번째 샘플의 유사도 행렬의 크기(shape) : (7, 7)


In [79]:
# 주어진 유사도 행렬로부터 그래프를 그려봅시다. >> 그래프는 안그려짐..

# def draw_graphs(sim_matrix):
#         nx_graph = nx.from_numpy_array(sim_matrix)
#         plt.figure(figsize=(10, 10))
#         pos = nx.spring_layout(nx_graph)
#         nx.draw(nx_graph, with_labels=True, font_weight='bold')
#         nx.draw_networkx_edge_labels(nx_graph,pos,font_color='red')
#         plt.show()

# draw_graphs(df['SimMatrix'][1])

이러한 그래프를 페이지랭크 알고리즘의 입력으로 사용하여 각 문장의 점수를 구합니다. 이렇게 'score'라는 열을 만듭니다.

In [172]:
# score계산이 안되는 기사들의 인덱스 찾는 코드
# for i in range(536,560):
#     print(i, calculate_score(df['SimMatrix'][i]))

# >> 254,451,535 인덱스를 가진 뉴스를 제거해야함 

# score 안구해지는 3개의 행 제거
new_df = df.drop([254,451,535])
len(new_df)

In [174]:
def calculate_score(sim_matrix):
    nx_graph = nx.from_numpy_array(sim_matrix)
    scores = nx.pagerank(nx_graph,max_iter=10000000)
    return scores

In [216]:
new_df['score'] = new_df['SimMatrix'].apply(calculate_score)
new_df[['SimMatrix', 'score']].head()

,SimMatrix,score
0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","{0: 0.020408657444630864, 1: 0.020408657444630..."
1,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0....","{0: 0.14285714285714282, 1: 0.1428571428571428..."
2,"[[1.0000000000000002, 0.0, 0.0, 0.0, 0.0, 1.00...","{0: 0.21729871298254208, 1: 0.0297029943837454..."
3,"[[0.9999998807907104, 0.0, 0.0, 0.736306428909...","{0: 0.2099685832684626, 1: 0.03333439569807714..."
4,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","{0: 0.04411960180587615, 1: 0.2940863711059815..."


두번째 샘플의 각 문장의 점수를 출력해봅시다. 총 7개 문장에 대해서 점수가 기록되어져 있습니다.

In [250]:
new_df['score'][1]

{0: 0.14285714285714282,
 1: 0.14285714285714282,
 2: 0.14285714285714282,
 3: 0.14285714285714282,
 4: 0.14285714285714282,
 5: 0.14285714285714282,
 6: 0.14285714285714282}

이제 이 점수가 가장 높은 문장들을 상위 n개 선택하여 이 문서의 요약문으로 삼을 것입니다. 점수가 가장 높은 상위 3개의 문장을 선택하는 함수를 만듭니다. 점수에 따라서 정렬 후에 상위 3개 문장만을 반환합니다.

In [176]:
def ranked_sentences(sentences, scores, n=3):
    top_scores = sorted(((scores[i],s) 
                         for i,s in enumerate(sentences)), 
                                reverse=True)
    top_n_sentences = [sentence 
                        for score,sentence in top_scores[:n]]
    return " ".join(top_n_sentences)

In [193]:
new_df['summary'] = new_df.apply(lambda x:ranked_sentences(x['sentences'], x['score']), axis=1)

기사 제목과 요약문 확인하기

In [261]:
for i in range(3):
    print(i,'번째', '\n', '기사 제목:', new_df['Title'][i], '\n', '요약:', new_df['summary'][i], '\n')

0 번째 
 기사 제목: '한달 전 데자뷔' 루나2 코인도 9분의 1토막…권도형 CEO '잠수'(종합) 
 요약: 비공개로 전환한 권도형 테라폼 랩스 최고경영자(CEO) 트위터[트위터 캡처. 미국 온라인 커뮤니티 레딧에서는 루나 2.0도 실패할 것으로 보면서 "루나 3.0이 조만간 나올 것"이라고 비꼬거나 "루나 2.0 가격이 급락하자 권 CEO가 트위터 계정을 비공개로 돌렸다"고 비판하는 의견이 나오고 있다. 권 CEO의 소재가 아직 알려지지 않은 가운데, 국내에서는 이달 들어 루나(LUNC) 투자로 손실을 본 투자자들이 권 CEO를 사기 등의 혐의로 잇따라 고소한 상태다. 

1 번째 
 기사 제목: “제2 루나 사태 막자”…코인마켓 거래소 공동 가이드라인 추진 
 요약: 이는 지난달 발생한 테라·루나 폭락 사태의 재발 방지하기 위해 추진된 계획이다. 이날 회의에는 ▶프로비트 ▶코어닥스 ▶플랫타익스체인지 ▶보라비트 ▶비트레이드 ▶BTX(구 비둘기) ▶빗크몬 ▶오아시스 등 8개 코인마켓 거래소와 김형중 고려대 정보보호대학원 특임교수, 권오훈(차앤권 법률사무소) 변호사 등이 참여했다. 또 일부 거래소의 독과점 해소, 은행 실명계좌 발급 확대 등을 위해서도 공동 대응하기로 했다. 

2 번째 
 기사 제목: 비트코인, 美 5월 CPI 발표 앞두고 하락세…3800만원대 
 요약:  기사내용 요약비트코인 3800만원대·이더리움 220만원대오는 10일 미국 5월 소비자물가지수 발표"반감기 앞둔 비트코인, 현재 매수 매력 높아"    [서울=뉴시스] 김제이 기자 = 비트코인이 미국의 5월 소비자물가지수(CPI) 발표를 코앞에 두고 관망세를 보이며 3800만원대에서 거래 중이다.9일 오전 8시42분 기준 국내 암호화폐 거래소 빗썸에서 비트코인은 24시간 전보다 2.60%내린 3854만6000원을 기록했다. 코인마켓캡에서는 1800달러를 기록했는데, 이는 24시간 전보다 0.46% 하락한 가격이다.비트코인을 비롯한 주요 메이저 코인들의 하락세는 오는 10일 발표될 미국 5

일부 문서에 대해서 원문과 요약문을 출력해볼까요?

In [262]:
for i in range(1):
    print(i,'번째', '\n', '기사 제목:', new_df['Title'][i], '\n', '\n', 
          
          '기사 원문:', new_df['Article'][i], '\n', '\n', 
          
          '요약:', new_df['summary'][i])

0 번째 
 기사 제목: '한달 전 데자뷔' 루나2 코인도 9분의 1토막…권도형 CEO '잠수'(종합) 
 
 기사 원문:  권씨 트위터 비공개 전환…"이제 루나 3.0 나오나" 비판도국내 투자자들 '사기 혐의' 고소…미국서도 스테이블코인 감독 강화    지난달 19일 루나(LUNC) 코인 급락 당시 시세 그래프[연합뉴스 자료사진](서울=연합뉴스) 차병섭 기자 = 가치 폭락으로 실패한 한국산 가상화폐 루나(LUNC)가 이른바 '루나 2.0'(LUNA)으로 가상화폐 시장에 돌아온 지 열흘을 넘긴 가운데 루나 2.0 역시 가격 급락세를 지속하고 있다.    9일 가상화폐 정보사이트 코인마켓캡에 따르면 루나 2.0은 한국시간 오후 4시 15분 기준 전일 대비 15.95% 하락한 2.93달러 선에서 거래되고 있으며, 최근 24시간 사이 2달러 선을 뚫고 1.96달러까지 떨어지기도 했다.    이 코인의 발행사인 테라폼 랩스에 따르면 루나 2.0의 시가총액은 한때 5억달러(약 6천278억원) 아래로 내려가는 등 최근 24시간 거래액이 시가총액을 앞섰다.    루나 2.0은 지난달 28일 오후 1개당 17.8달러에 외국 주요 거래소에 상장됐고, 상장 직후 19.54달러까지 올랐다가 4달러대로 수직 추락하는 등 급격한 변동성을 보였다.    지난달 31일 다시 10달러를 넘기도 했지만 이후 별다른 반등 없이 계단식 하락을 계속하면서 이날 신저가 1.96달러는 상장 당시 가격의 11%에 불과한 수준이다.     지난달 10일께 루나(LUNC)의 자매 코인인 스테이블 코인 테라USD(UST)가 기준 가격인 1달러 밑으로 떨어지면서 세계적인 투매 행진이 시작된 지 약 한 달 만으로, 이번에는 루나 2.0 가격 급락으로 테라 블록체인 '부활'의 꿈이 멀어지는 분위기다.    원조 루나(LUNC)도 최근 24시간 사이 최저·최고가가 0.00004352∼0.00008259달러를 오가는 등 급등락하고 있다.    비공개로 전환한 권도형 테라폼 랩스 최고경영자(CEO) 트위터[트위터 캡

In [252]:
file_path =  "./Data/"

file_name = "네이버_뉴스_" + '코인_기사별_텍스트요약' + ".csv"

file_full_path = file_path + file_name

new_df.to_csv(file_full_path ,  index = False, encoding="utf-8-sig") # utf-8-sig로 해야함! ㅜto_csv(file_full_path ,  index = False, encoding="utf-8-sig") # utf-8-sig로 해야함! to_csv(file_full_path ,  index = False, encoding="utf-8-sig") # utf-8-sig로 해야함! ㅜto_csv(file_full_path ,  index = False, encoding="utf-8-sig") # utf-8-sig로 해야함! ㅜ 